# IE 582 Fall 2019 -  Project - Group ID 10
### Members:
Günay Eser

**Importing necessary libraries:**

In [183]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
import statistics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import  Lasso
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.tree import DecisionTreeRegressor

**Droping unnecessary data and give each column a new name for the sake of simplicity:**

In [184]:
matches = pd.read_csv('matches.csv')
matches = matches.drop(['epoch', 'match_status', 'match_live', 'match_hometeam_penalty_score' , 'match_awayteam_penalty_score',  'match_hometeam_extra_score', 'match_awayteam_extra_score'], axis=1)
matches.columns = ['away_id', 'home_id', 'match_id', 'home_name', 'away_name' ,  'home_score', 'away_score', 'home_half_score', 'away_half_score', 'league_id']
matches.head()

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id
0,7109,7097,41196,Levante,Valencia,1.0,1.0,1.0,1.0,468
1,2614,2619,13331,Crystal Palace,Southampton,0.0,1.0,0.0,1.0,148
2,3224,3238,17683,Eintracht Frankfurt,FC Augsburg,1.0,2.0,0.0,1.0,195
3,3235,3223,17684,SV Werder Bremen,Schalke,1.0,2.0,1.0,1.0,195
4,3237,3225,17682,Bayern Munich,1. FSV Mainz 05,4.0,0.0,2.0,0.0,195


**Null values are needed to be changed into empty string in order to filter them more easily:**

In [185]:
matches.fillna('', inplace = True) 

**Last 20 rows from the data with English Premier League ID, the ones with null values are tha upcoming games**

In [186]:
matches[matches['league_id'] == 148].tail(20)

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id
6004,2615,2613,273292,Brighton,Bournemouth,2,0,1,0,148
6009,2612,2630,273296,Newcastle,Everton,1,2,0,1,148
6010,2619,2614,273298,Southampton,Crystal Palace,1,1,0,0,148
6011,2632,2623,273299,Watford,Aston Villa,3,0,1,0,148
6014,2611,2620,273300,West Ham,Leicester,1,2,1,1,148
6015,2628,2641,273297,Norwich,Tottenham,2,2,1,0,148
6016,2627,2629,273293,Burnley,Manchester Utd,0,2,0,1,148
6017,2654,2626,273295,Manchester City,Sheffield Utd,,,,,148
6019,2616,2613,273302,Brighton,Chelsea,,,,,148
6020,2632,2629,273303,Burnley,Aston Villa,,,,,148


**Let's just extract only the matches from English Premier League**

In [187]:
matches_gb = matches[matches['league_id'] == 148].copy()
matches_gb = matches_gb.reset_index(drop=True)

**One-Hot Encoding**

In [188]:
matches_gb['home_id'] = pd.Categorical(matches_gb['home_id'])
dfDummies = pd.get_dummies(matches_gb['home_id'], prefix = 'home_id')
matches_gb = pd.concat([matches_gb, dfDummies], axis=1)

matches_gb['away_id'] = pd.Categorical(matches_gb['away_id'])
dfDummies = pd.get_dummies(matches_gb['away_id'], prefix = 'away_id')
matches_gb = pd.concat([matches_gb, dfDummies], axis=1)

**Extracting the upcoming games into another dataframe**

In [189]:
cut = matches_gb[matches_gb['home_score'] == ''].index[0]
coming_games = matches_gb.iloc[cut:,:].copy().reset_index(drop=True)
coming_games

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
0,2654,2626,273295,Manchester City,Sheffield Utd,,,,,148,...,0,0,0,0,0,0,0,0,0,1
1,2616,2613,273302,Brighton,Chelsea,,,,,148,...,0,0,0,0,0,0,0,0,0,0
2,2632,2629,273303,Burnley,Aston Villa,,,,,148,...,0,0,0,0,0,0,1,0,0,0
3,2611,2630,273306,Newcastle,Leicester,,,,,148,...,0,0,0,0,0,0,0,0,0,0
4,2628,2614,273308,Southampton,Tottenham,,,,,148,...,0,0,0,1,0,0,0,0,0,0
5,2646,2623,273309,Watford,Wolves,,,,,148,...,0,0,0,0,0,0,0,0,1,0
6,2612,2626,273305,Manchester City,Everton,,,,,148,...,0,0,0,0,0,0,0,0,0,0
7,2615,2620,273310,West Ham,Bournemouth,,,,,148,...,0,0,0,0,0,0,0,0,0,0
8,2619,2641,273307,Norwich,Crystal Palace,,,,,148,...,0,0,0,0,0,0,0,0,0,0
9,2627,2617,273301,Arsenal,Manchester Utd,,,,,148,...,0,0,1,0,0,0,0,0,0,0


In [190]:
matches_gb = matches_gb.iloc[:cut,:]

In [191]:
matches_gb.tail()

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
734,2619,2614,273298,Southampton,Crystal Palace,1,1,0,0,148,...,0,0,0,0,0,0,0,0,0,0
735,2632,2623,273299,Watford,Aston Villa,3,0,1,0,148,...,0,0,0,0,0,0,1,0,0,0
736,2611,2620,273300,West Ham,Leicester,1,2,1,1,148,...,0,0,0,0,0,0,0,0,0,0
737,2628,2641,273297,Norwich,Tottenham,2,2,1,0,148,...,0,0,0,1,0,0,0,0,0,0
738,2627,2629,273293,Burnley,Manchester Utd,0,2,0,1,148,...,0,0,1,0,0,0,0,0,0,0


**Now let's add a column to indicate the winner, or tie in case there is no winner:**

In [192]:
homes = matches_gb.iloc[:,5:6].values.tolist()
aways = matches_gb.iloc[:,6:7].values.tolist()
away_teams = matches_gb.iloc[:,0:1].values.tolist()
home_teams = matches_gb.iloc[:,1:2].values.tolist()

In [193]:
winners = list()
for i in range(len(homes)):
    if homes[i][0] > aways[i][0]:
        winners.append(str(home_teams[i][0]))
        
    elif homes[i][0] == aways[i][0]:
        winners.append("Tie")
        
    elif homes[i][0] < aways[i][0]:
        winners.append(winners.append(str(away_teams[i][0])))

In [194]:
for i in winners:
    if i == None:
        winners.remove(i)

In [195]:
matches_gb['winners'] = winners

In [196]:
matches_gb[['away_id','home_id', 'winners']].head()

,away_id,home_id,winners
0,2614,2619,2614
1,2626,2623,2626
2,2629,2621,Tie
3,2641,2654,2641
4,2617,2616,Tie


**I would like our model to know the stronger teams, so I would create a new function to calculate each teams points by number of wins and scale these points between 1 to 10**

In [197]:
teams = matches_gb.home_id.unique()

In [198]:
teams_list = matches_gb.home_id.unique().astype(str).tolist()

In [199]:
def calculate_team_points(match_id):
    team_points = dict()
    for i in teams_list:
        team_points[i] = 0
        
    index = index = matches_gb[matches_gb['match_id'] == match_id ].index[0]
    games = matches_gb.loc[:index,].copy()
    winners2 = games.winners.tolist()
    
    for i in teams_list:
        team_points[i] = winners.count(i)
    
    table = pd.DataFrame(zip(team_points.keys(), team_points.values()), columns = ["teams", 'points'])
    table = table.sort_values(by = 'points', ascending= False).reset_index(drop=True)
    
    x = table[['points']].values.astype(float)
    
    
    # Scaling points between 1 - 10
    news = list()
    for k in x:
        rng = max(x)[0] - min(x)[0] 
        new = 1 + abs(k -  min(x)[0])*(9/rng)
        news.append(new[0])
        
   
    team_points_test = table.copy()
    team_points_test['points'] = news
    return team_points_test
    

**Reading stats.csv file**

In [200]:
stats = pd.read_csv('stats.csv')
stats.head()

,match_id,home_BallPossession,home_CornerKicks,home_Fouls,home_GoalAttempts,home_GoalkeeperSaves,home_Offsides,home_ShotsoffGoal,home_ShotsonGoal,home_YellowCards,...,home_DistanceCovered(metres),away_DistanceCovered(metres),home_PassSuccess%,away_PassSuccess%,home_Attacks,home_DangerousAttacks,away_Attacks,away_DangerousAttacks,home_,away_
0,13327,71%,12.0,7.0,35.0,3.0,1.0,14.0,9.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13329,33%,3.0,8.0,7.0,4.0,2.0,5.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13331,45%,5.0,14.0,14.0,3.0,0.0,9.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13446,51%,7.0,7.0,6.0,3.0,0.0,4.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13447,49%,3.0,18.0,20.0,1.0,1.0,8.0,6.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Let's just extract the features we need:**

In [201]:
stats = stats.iloc[:,[0, 1,7,8,10,16,17,28,30]].copy()

In [202]:
stats.head()

,match_id,home_BallPossession,home_ShotsoffGoal,home_ShotsonGoal,away_BallPossession,away_ShotsoffGoal,away_ShotsonGoal,home_TotalPasses,away_TotalPasses
0,13327,71%,14.0,9.0,29%,1.0,4.0,676.0,277.0
1,13329,33%,5.0,1.0,67%,10.0,10.0,312.0,637.0
2,13331,45%,9.0,3.0,55%,4.0,4.0,398.0,494.0
3,13446,51%,4.0,2.0,49%,5.0,6.0,NaN,NaN
4,13447,49%,8.0,6.0,51%,5.0,2.0,NaN,NaN


**Reading bets.csv file and extract only the result odds of bookmakers:**

In [203]:
bets = pd.read_csv('bets.csv')
bets = bets[(bets.variable == 'odd_1') | (bets.variable == 'odd_2') | (bets.variable == 'odd_x')]

In [204]:
bets_table = pd.pivot_table(bets.loc[:,['match_id', 'variable','value']], index=['match_id'], columns='variable')
bets_table['match_id'] = bets_table.index
bets_table = bets_table.reset_index(drop=True)
bets_table.head(10)

value                     match_id
variable     odd_1     odd_2     odd_x         
0         6.039825  1.387018  4.721579   145899
1         2.109828  2.843966  3.616724   145900
2         1.960000  3.650000  3.490000   146845
3         1.995714  3.397143  3.092857   147984
4         2.135294  3.815294  3.245294   147990
5         1.920588  4.316471  3.483529   147991
6         8.396471  1.395882  4.920588   147992
7         8.856471  1.406471  4.657647   147993
8         2.904118  2.427059  3.509412   147995
9         1.765294  5.165294  3.592941   147996

In [205]:
merged = pd.merge(matches_gb, bets_table, on='match_id').rename({'(value,odd_1)':'odd_1'})

In [206]:
merged.columns = ['away_id',          'home_id',         'match_id',
              'home_name',        'away_name',       'home_score',
             'away_score',  'home_half_score',  'away_half_score',
              'league_id',     'home_id_2611',     'home_id_2612',
           'home_id_2613',     'home_id_2614',     'home_id_2615',
           'home_id_2616',     'home_id_2617',     'home_id_2619',
           'home_id_2620',     'home_id_2621',     'home_id_2623',
           'home_id_2626',     'home_id_2627',     'home_id_2628',
           'home_id_2629',     'home_id_2630',     'home_id_2632',
           'home_id_2641',     'home_id_2646',     'home_id_2654',
           'away_id_2611',     'away_id_2612',     'away_id_2613',
           'away_id_2614',     'away_id_2615',     'away_id_2616',
           'away_id_2617',     'away_id_2619',     'away_id_2620',
           'away_id_2621',     'away_id_2623',     'away_id_2626',
           'away_id_2627',     'away_id_2628',     'away_id_2629',
           'away_id_2630',     'away_id_2632',     'away_id_2641',
           'away_id_2646',     'away_id_2654',          'winners', 'odd_1',
       'odd_2', 'odd_x']

In [207]:
merged['odd_1'] = 1/merged['odd_1']
merged['odd_2'] = 1/merged['odd_2']
merged['odd_x'] = 1/merged['odd_x']
merged.head()

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654,winners,odd_1,odd_2,odd_x
0,2612,2616,150842,Chelsea,Everton,0,0,0,0,148,...,0,0,0,0,0,0,Tie,0.194175,0.568182,0.270270
1,2627,2626,150844,Manchester City,Manchester Utd,3,1,1,0,148,...,0,0,0,0,0,0,2626,0.343249,0.410959,0.303030
2,2615,2626,156848,Manchester City,Bournemouth,3,1,1,1,148,...,0,0,0,0,0,0,2626,0.370430,0.375756,0.340690
3,2623,2611,156847,Leicester,Watford,2,0,2,0,148,...,0,0,0,0,0,0,2611,0.588560,0.202962,0.289960
4,2629,2619,156845,Crystal Palace,Burnley,2,0,1,0,148,...,1,0,0,0,0,0,2619,0.432232,0.305744,0.345338


In [208]:
matches_stats = pd.merge(merged[['winners','match_id','home_id', 'away_id', 'odd_1', 'odd_2', 'odd_x', 'home_id_2611', 'home_id_2612', 'home_id_2613',
       'home_id_2614', 'home_id_2615', 'home_id_2616', 'home_id_2617',
       'home_id_2619', 'home_id_2620', 'home_id_2621', 'home_id_2623',
       'home_id_2626', 'home_id_2627', 'home_id_2628', 'home_id_2629',
       'home_id_2630', 'home_id_2632', 'home_id_2641', 'home_id_2646',
       'home_id_2654', 'away_id_2611', 'away_id_2612', 'away_id_2613',
       'away_id_2614', 'away_id_2615', 'away_id_2616', 'away_id_2617',
       'away_id_2619', 'away_id_2620', 'away_id_2621', 'away_id_2623',
       'away_id_2626', 'away_id_2627', 'away_id_2628', 'away_id_2629',
       'away_id_2630', 'away_id_2632', 'away_id_2641', 'away_id_2646',
       'away_id_2654'] ],stats, on='match_id')# .isnull().sum()

### Rows with null values

In [209]:
matches_stats[matches_stats.isna().any(axis=1)]

,winners,match_id,home_id,away_id,odd_1,odd_2,odd_x,home_id_2611,home_id_2612,home_id_2613,...,away_id_2646,away_id_2654,home_BallPossession,home_ShotsoffGoal,home_ShotsonGoal,away_BallPossession,away_ShotsoffGoal,away_ShotsonGoal,home_TotalPasses,away_TotalPasses
53,2613,164854,2615,2613,0.474291,0.269460,0.323341,0,0,0,...,0,0,49%,6.0,3.0,51%,5.0,3.0,NaN,NaN
191,2646,237422,2630,2646,0.306290,0.398287,0.292708,0,0,0,...,1,0,42%,2.0,3.0,58%,2.0,6.0,NaN,NaN
192,2626,237423,2626,2620,0.494321,0.261518,0.284728,0,0,0,...,0,0,55%,1.0,10.0,45%,4.0,5.0,NaN,NaN
193,2630,267786,2630,2620,0.262871,0.540113,0.269595,0,0,0,...,0,0,42%,4.0,2.0,58%,4.0,0.0,NaN,NaN
310,2623,224540,2641,2623,0.416034,0.305573,0.264240,0,0,0,...,0,0,66%,7.0,5.0,34%,6.0,2.0,NaN,NaN
356,2632,327989,2632,2621,0.782163,0.132068,0.151427,0,0,0,...,0,0,43%,3.0,8.0,57%,6.0,8.0,NaN,NaN


Our function to calculate team stats:

In [210]:
def get_team_rates(match_id, team_id, order, new = False):
    
    if new == False:
        index = matches_gb[matches_gb['match_id'] == match_id ].index[0]
        games_played = matches_gb.loc[:index,].copy()
    if new == True:
        games_played = matches_gb.copy()
    
    
    
    games_played = games_played[(games_played.away_id == team_id) | (games_played.home_id == team_id)].reset_index(drop=True)
    
    home_games = matches_stats[(matches_stats.home_id == team_id)]
    away_games = matches_stats[(matches_stats.away_id == team_id)]
    
    total_win_rate = games_played.winners.tolist().count(str(team_id)) / len(games_played['winners'])
    home_win_rate = games_played[games_played['home_id'] == team_id].winners.tolist().count(str(team_id)) / len(games_played[games_played['home_id'] == team_id]['winners'])
    away_win_rate = games_played[games_played['away_id'] == team_id].winners.tolist().count(str(team_id)) / len(games_played[games_played['away_id'] == team_id]['winners'])
    
    last6_games = games_played.tail(6).copy()
    last6_home_games = last6_games[last6_games['home_id'] == team_id ] # Home Games in Last 6 matches
    last6_away_games = last6_games[last6_games['away_id'] == team_id ] # Away Games in Last 6 matches
    
    oldest_match = last6_games.match_id.values[0]
    
    team_pts = calculate_team_points(oldest_match)  # Points of all teams until the given match.
    
    goals_conceded_home = sum(last6_home_games.away_score.tolist())
    goals_conceded_away = sum(last6_home_games.home_score.tolist())
    
    total_goals_conceded = goals_conceded_away + goals_conceded_home
    
    mean_conceded_goals = statistics.mean(last6_home_games.away_score.tolist() + last6_home_games.home_score.tolist())
    
    
    home_goals = last6_home_games.home_score.tolist()
    home_goals = np.asarray(home_goals)
    
    away_goals = last6_away_games.away_score.tolist()
    away_goals = np.asarray(away_goals)
    

    home_points = list()
    for i in last6_home_games.away_id.tolist():
        home_points.append(team_pts[team_pts['teams'] == str(i)]['points'].tolist()[0])

    home_points = np.asarray(home_points)
    home_score_points = home_points*home_goals
    
    away_points = list()
    for i in last6_away_games.home_id.tolist():
        away_points.append(team_pts[team_pts['teams'] == str(i)]['points'].tolist()[0])
        
    away_points = np.asarray(away_points)
    away_score_points = away_points*away_goals
    
    score_points = sum(home_score_points) + sum(away_score_points)                      # Score Points
    
    stdev_of_goals = statistics.stdev(list(home_goals) + list(away_goals))              # Standart Deviation of Goals
    
    if new == False:
        games_played = matches_stats.loc[:index,].copy()
    else:
        games_played = matches_stats.copy()
    
    home_possessions = home_games['home_BallPossession'].tail(6).values.tolist()
    summation = 0
    for j in home_possessions:
        summation += int(j.strip('%'))
    
    home_mean_possession = summation / len(home_possessions)   # Mean Ball Possession in  Last 6 Home Matches
        
    away_possessions = away_games['away_BallPossession'].tail(6).values.tolist()
    summation = 0
    for j in away_possessions:
        summation += int(j.strip('%'))
    
    away_mean_possession = summation / len(away_possessions)   # Mean Ball Possession in  Last 6 Away Matches
    
    
    home_shots_on = home_games['home_ShotsonGoal'].tail(6).values.tolist()
    mean_home_shots_on = statistics.mean(home_shots_on)
    away_shots_on = away_games['away_ShotsonGoal'].tail(6).values.tolist()
    mean_away_shots_on = statistics.mean(away_shots_on)
    
    home_shots_off = home_games['home_ShotsoffGoal'].tail(6).values.tolist()
    away_shots_off = away_games['away_ShotsoffGoal'].tail(6).values.tolist()
        
    shots_rate = sum(np.asarray(home_shots_on + away_shots_on)) / sum(np.asarray(home_shots_on + home_shots_off + away_shots_on + away_shots_off))
    
    
    
    if order == 1:
        value = {'total_win_rate': total_win_rate, 'home_win_rate': home_win_rate, 'away_win_rate': away_win_rate, 'away_goals': sum(away_goals), 'home_goals': sum(home_goals), 'score_points': score_points, 'goals_conceded': total_goals_conceded, 'mean_conceded': mean_conceded_goals, 'stdev_goals':stdev_of_goals, 'h_mean_poss': home_mean_possession, 'a_mean_poss': away_mean_possession, 'h_mean_shotson':  mean_home_shots_on, 'a_mean_shotson':mean_away_shots_on, 'shots_rate':shots_rate }
    else:
        value = {'total_win_rate2': total_win_rate, 'home_win_rate2': home_win_rate, 'away_win_rate2': away_win_rate, 'away_goals2': sum(away_goals), 'home_goals2': sum(home_goals), 'score_points2': score_points, 'goals_conceded2': total_goals_conceded, 'mean_conceded2': mean_conceded_goals, 'stdev_goals2':stdev_of_goals, 'h_mean_poss2': home_mean_possession, 'a_mean_poss2': away_mean_possession, 'h_mean_shotson2':  mean_home_shots_on, 'a_mean_shotson2':mean_away_shots_on, 'shots_rate2':shots_rate }
    
    return value
    
# Data to Return:  Total Win Rate, Home Win Rate, Away Win Rate, H. Goals, A. Goals, Score Points, Goals Conceded, Mean Goals Conceded, Stdev of Goals, H Mean Poss, A Mean Poss, mean H.ShotsOn, mean A.ShotsOn, ShotsRate
# Score Points: is calculated by multiplying each scored goal quantities for each match with the Point of the Opponent Team.
    
    

<hr>

In [211]:
matches_stats.head()

,winners,match_id,home_id,away_id,odd_1,odd_2,odd_x,home_id_2611,home_id_2612,home_id_2613,...,away_id_2646,away_id_2654,home_BallPossession,home_ShotsoffGoal,home_ShotsonGoal,away_BallPossession,away_ShotsoffGoal,away_ShotsonGoal,home_TotalPasses,away_TotalPasses
0,Tie,150842,2616,2612,0.194175,0.568182,0.270270,0,0,0,...,0,0,69%,7.0,4.0,31%,3.0,1.0,688.0,311.0
1,2626,150844,2626,2627,0.343249,0.410959,0.303030,0,0,0,...,0,0,65%,6.0,5.0,35%,1.0,1.0,703.0,380.0
2,2626,156848,2626,2615,0.370430,0.375756,0.340690,0,0,0,...,0,0,73%,4.0,6.0,27%,2.0,1.0,786.0,287.0
3,2611,156847,2611,2623,0.588560,0.202962,0.289960,1,0,0,...,0,0,38%,3.0,3.0,62%,6.0,0.0,304.0,498.0
4,2619,156845,2619,2629,0.432232,0.305744,0.345338,0,0,0,...,0,0,58%,11.0,9.0,42%,2.0,0.0,519.0,356.0


In [212]:
 matches_stats.tail()

,winners,match_id,home_id,away_id,odd_1,odd_2,odd_x,home_id_2611,home_id_2612,home_id_2613,...,away_id_2646,away_id_2654,home_BallPossession,home_ShotsoffGoal,home_ShotsonGoal,away_BallPossession,away_ShotsoffGoal,away_ShotsonGoal,home_TotalPasses,away_TotalPasses
378,Tie,273298,2614,2619,0.505844,0.259373,0.283935,0,0,0,...,0,0,60%,4.0,5.0,40%,3.0,2.0,474.0,332.0
379,2623,273299,2623,2632,0.505532,0.276692,0.268112,0,0,0,...,0,0,48%,4.0,6.0,52%,3.0,2.0,412.0,430.0
380,2611,273300,2620,2611,0.256032,0.484903,0.255767,0,0,0,...,0,0,43%,5.0,4.0,57%,2.0,8.0,363.0,496.0
381,Tie,273297,2641,2628,0.198477,0.612870,0.236869,0,0,0,...,0,0,41%,3.0,2.0,59%,1.0,7.0,383.0,552.0
382,2627,273293,2629,2627,0.238294,0.535601,0.272763,0,0,0,...,0,0,38%,5.0,1.0,62%,3.0,5.0,289.0,502.0


In [213]:
rows = list()
for i in matches_stats.match_id.index:
    match_id = matches_stats.at[i,'match_id']
    home_id = matches_stats.at[i,'home_id']
    away_id = matches_stats.at[i,'away_id']
    #print(home_id)
    
    
    
    winner = {'winner': 0}
    win = {'home_win' : 0 , 'away_win' : 0, 'tie' : 0}
    
    if matches_stats.at[i,'winners'] == str(home_id):
        winner['winner'] = 1
        win['home_win'] = 1
    elif matches_stats.at[i, 'winners'] == str(away_id):
        winner['winner'] = 3
        win['away_win'] = 1
    else:
        winner['winner'] = 2
        win['tie'] = 1
        
    temp = {'match_id':match_id, 'home_id':home_id, 'away_id':away_id, 'odd_1':matches_stats.at[i,'odd_1'], 'odd_2':matches_stats.at[i,'odd_2'], 'odd_x':matches_stats.at[i,'odd_x']}
    
    winner.update(temp)
    
    dt = get_team_rates(match_id = match_id, team_id = home_id, order = 1)
    winner.update(dt)
    
    dt2 = get_team_rates(match_id = match_id, team_id = away_id , order = 2)
    winner.update(dt2)
    
    rows.append(winner)
    
    
df  = pd.DataFrame(rows)
df.head(10)

,winner,match_id,home_id,away_id,odd_1,odd_2,odd_x,total_win_rate,home_win_rate,away_win_rate,...,home_goals2,score_points2,goals_conceded2,mean_conceded2,stdev_goals2,h_mean_poss2,a_mean_poss2,h_mean_shotson2,a_mean_shotson2,shots_rate2
0,2,150842,2616,2612,0.194175,0.568182,0.270270,0.488889,0.538462,0.421053,...,5.0,32.75,6.0,1.500000,1.211060,50.833333,43.333333,4.666667,5.000000,0.479339
1,1,150844,2626,2627,0.343249,0.410959,0.303030,0.769231,0.857143,0.666667,...,5.0,49.55,8.0,2.000000,0.752773,52.833333,51.166667,7.000000,5.333333,0.569231
2,1,156848,2626,2615,0.370430,0.375756,0.340690,0.780488,0.863636,0.684211,...,4.0,27.30,9.0,1.125000,0.632456,44.000000,54.000000,2.333333,3.000000,0.438356
3,1,156847,2611,2623,0.588560,0.202962,0.289960,0.324324,0.400000,0.235294,...,0.0,10.65,7.0,1.750000,0.836660,40.500000,37.333333,3.166667,1.666667,0.420290
4,1,156845,2619,2629,0.432232,0.305744,0.345338,0.200000,0.285714,0.105263,...,1.0,9.30,7.0,1.750000,0.836660,38.500000,39.000000,4.666667,1.500000,0.445783
5,2,156850,2614,2627,0.567334,0.212101,0.302934,0.142857,0.157895,0.130435,...,2.0,41.70,3.0,0.750000,0.836660,52.833333,51.166667,7.000000,5.333333,0.569231
6,1,157536,2617,2628,0.306538,0.461496,0.302019,0.512195,0.695652,0.277778,...,3.0,53.70,5.0,1.250000,1.264911,55.833333,50.500000,3.500000,4.333333,0.540230
7,1,157537,2621,2612,0.774154,0.103383,0.192520,0.658537,0.684211,0.636364,...,5.0,32.75,6.0,1.500000,1.211060,50.833333,43.333333,4.666667,5.000000,0.479339
8,1,158697,2613,2619,0.877893,0.057569,0.114520,0.300000,0.454545,0.111111,...,4.0,26.10,7.0,1.166667,0.894427,39.833333,48.000000,3.000000,2.166667,0.382716
9,3,158699,2623,2626,0.614704,0.181418,0.250912,0.297297,0.421053,0.166667,...,12.0,69.55,15.0,2.500000,1.722401,66.166667,62.833333,7.166667,6.833333,0.531646


In [93]:
df = pd.merge(df,matches_stats , on =['match_id', 'home_id', 'away_id', 'odd_1', 'odd_2', 'odd_x']).drop(['winners','home_BallPossession', 'home_ShotsoffGoal', 'home_ShotsonGoal',
       'away_BallPossession', 'away_ShotsoffGoal', 'away_ShotsonGoal',
       'home_TotalPasses', 'away_TotalPasses' ], axis=1)

The final dataframe to be put in our model:

In [97]:
df['h_mean_poss'] = df['h_mean_poss']/100
df['h_mean_poss2'] = df['h_mean_poss2']/100
df['a_mean_poss'] = df['a_mean_poss']/100
df['a_mean_poss2'] = df['a_mean_poss2']/100
df.head()

,winner,match_id,home_id,away_id,odd_1,odd_2,odd_x,total_win_rate,home_win_rate,away_win_rate,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
0,2,150842,2616,2612,0.194175,0.568182,0.270270,0.488889,0.538462,0.421053,...,0,0,0,0,0,0,0,0,0,0
1,1,150844,2626,2627,0.343249,0.410959,0.303030,0.769231,0.857143,0.666667,...,0,0,1,0,0,0,0,0,0,0
2,1,156848,2626,2615,0.370430,0.375756,0.340690,0.780488,0.863636,0.684211,...,0,0,0,0,0,0,0,0,0,0
3,1,156847,2611,2623,0.588560,0.202962,0.289960,0.324324,0.400000,0.235294,...,1,0,0,0,0,0,0,0,0,0
4,1,156845,2619,2629,0.432232,0.305744,0.345338,0.200000,0.285714,0.105263,...,0,0,0,0,1,0,0,0,0,0


Define our function to calculate RPS score:

In [149]:
def rps(p_home, p_draw, p_away, result):
    probs = [p_home, p_draw, p_away]
    result_p = [0,0,0]
    result_p[result-1] = 1
    rps_sum = 0
    for i in range(1,4):
        sum_p = 0
        for k in range(i):
            sum_p += probs[k]
        sum_r = 0
        for j in range(i):
            sum_r += result_p[j]
        rps_sum += (sum_p - sum_r)**2
    return rps_sum/2

### Logistic Regresion

**Parameter tuning for logistic regression using 10 fold cross-validation:**

In [175]:
rps_scores = list()
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,4:], df.iloc[:,[0]], test_size = 0.15)
    y_reals = y_test.values.ravel()
    penalty = ['l1', 'l2']
    C = [0.001, 0.01, 0.1, 1, 10, 100, 1000] 
    hyperparameters = dict(C=C, penalty=penalty)
    logy = LogisticRegression(multi_class = 'multinomial', solver = 'saga')

    clf = GridSearchCV(logy, hyperparameters, cv=10, verbose=0)
    best_model = clf.fit(x_train, y_train.values.ravel())


    print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
    print('Best C:', best_model.best_estimator_.get_params()['C'])
    y_pred = best_model.predict_proba(x_test)
    y_preds = best_model.predict(x_test)
    print("Accuracy : " , best_model.score(x_test,y_test))
    rpses = list()
    for i in range(len(y_reals)):

        rpses.append(rps(y_pred[i][0], y_pred[i][1], y_pred[i][2], y_reals[i]))
        #print(rps(y_pred[i][0], y_pred[i][1], y_pred[i][2], y_reals[i]))

    print('RPS Score: ',statistics.mean(rpses))
    rps_scores.append(statistics.mean(rpses))

Best Penalty: l2
Best C: 0.01
Accuracy :  0.6206896551724138
RPS Score:  0.17158063230042597
Best Penalty: l1
Best C: 0.1
Accuracy :  0.603448275862069
RPS Score:  0.15089869065597575
Best Penalty: l2
Best C: 100
Accuracy :  0.7241379310344828
RPS Score:  0.10971506468930081
Best Penalty: l1
Best C: 0.1
Accuracy :  0.6206896551724138
RPS Score:  0.18230525225950397
Best Penalty: l1
Best C: 1
Accuracy :  0.6551724137931034
RPS Score:  0.15197681544762726
Best Penalty: l1
Best C: 0.1
Accuracy :  0.7068965517241379
RPS Score:  0.14542813907487892
Best Penalty: l1
Best C: 1
Accuracy :  0.6206896551724138
RPS Score:  0.13746278150739197
Best Penalty: l2
Best C: 0.01
Accuracy :  0.6896551724137931
RPS Score:  0.12983389880879134
Best Penalty: l2
Best C: 0.1
Accuracy :  0.6724137931034483
RPS Score:  0.15857497000911852
Best Penalty: l2
Best C: 1
Accuracy :  0.6551724137931034
RPS Score:  0.15837313613683457


These are the results for 10 times 0.15 test data predictions.

In [177]:
statistics.mean(rps_scores)

0.1496149380889849

In [165]:
best_model.fit(df.iloc[:,4:], df.iloc[:,[0]])

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [166]:
y_pred  # P(Home) , P(Tie) , P(Away)

array([[0.5428133 , 0.28282582, 0.17436088],
       [0.56046565, 0.23398284, 0.20555151],
       [0.53066356, 0.30924426, 0.16009217],
       [0.42589245, 0.34064857, 0.23345898],
       [0.60841334, 0.23835343, 0.15323323],
       [0.28697935, 0.20498944, 0.50803121],
       [0.09813572, 0.21607045, 0.68579384],
       [0.66027197, 0.14630627, 0.19342176],
       [0.43154537, 0.24130686, 0.32714777],
       [0.60467963, 0.19843396, 0.19688641],
       [0.52334089, 0.31131413, 0.16534498],
       [0.36311627, 0.22851397, 0.40836975],
       [0.82006796, 0.12235706, 0.05757498],
       [0.02307439, 0.07068557, 0.90624004],
       [0.57046978, 0.20530757, 0.22422265],
       [0.2430246 , 0.22729107, 0.52968432],
       [0.31755628, 0.27477055, 0.40767317],
       [0.83688511, 0.12209563, 0.04101926],
       [0.03485542, 0.0645838 , 0.90056078],
       [0.88279721, 0.06098413, 0.05621866],
       [0.20486227, 0.25717624, 0.5379615 ],
       [0.81887588, 0.13424476, 0.04687936],
       [0.

### Upcoming Matches

Dataframe of the upcoming games would be:

In [167]:
coming_games

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
0,2654,2626,273295,Manchester City,Sheffield Utd,,,,,148,...,0,0,0,0,0,0,0,0,0,1
1,2616,2613,273302,Brighton,Chelsea,,,,,148,...,0,0,0,0,0,0,0,0,0,0
2,2632,2629,273303,Burnley,Aston Villa,,,,,148,...,0,0,0,0,0,0,1,0,0,0
3,2611,2630,273306,Newcastle,Leicester,,,,,148,...,0,0,0,0,0,0,0,0,0,0
4,2628,2614,273308,Southampton,Tottenham,,,,,148,...,0,0,0,1,0,0,0,0,0,0
5,2646,2623,273309,Watford,Wolves,,,,,148,...,0,0,0,0,0,0,0,0,1,0
6,2612,2626,273305,Manchester City,Everton,,,,,148,...,0,0,0,0,0,0,0,0,0,0
7,2615,2620,273310,West Ham,Bournemouth,,,,,148,...,0,0,0,0,0,0,0,0,0,0
8,2619,2641,273307,Norwich,Crystal Palace,,,,,148,...,0,0,0,0,0,0,0,0,0,0
9,2627,2617,273301,Arsenal,Manchester Utd,,,,,148,...,0,0,1,0,0,0,0,0,0,0


## Predicting Next Matches

We already have our dataframe named **coming_games**

In [168]:
results = str()
rows2 = list()
for i in range(len(coming_games.match_id.values.tolist())):
    match_id = coming_games.at[i,'match_id']
    home_id = coming_games.at[i,'home_id']
    away_id = coming_games.at[i,'away_id']
    
    temp2 = {'match_id':match_id, 'home_id':home_id, 'away_id':away_id, 'odd_1':matches_stats.at[i,'odd_1'], 'odd_2':matches_stats.at[i,'odd_2'], 'odd_x':matches_stats.at[i,'odd_x']}
    
    first = get_team_rates(match_id, home_id, order = 1, new = True)
    second = get_team_rates(match_id, away_id, order = 2, new = True)
    
    temp2.update(first)
    temp2.update(second)
    
    rows2.append(temp2)
    
df2  = pd.DataFrame(rows2)
   
df2.head()

,match_id,home_id,away_id,odd_1,odd_2,odd_x,total_win_rate,home_win_rate,away_win_rate,away_goals,...,home_goals2,score_points2,goals_conceded2,mean_conceded2,stdev_goals2,h_mean_poss2,a_mean_poss2,h_mean_shotson2,a_mean_shotson2,shots_rate2
0,273295,2626,2654,0.194175,0.568182,0.270270,0.761364,0.851064,0.658537,11.0,...,3.0,15.25,6.0,1.000000,0.752773,45.833333,40.333333,4.833333,2.833333,0.469388
1,273302,2613,2616,0.343249,0.410959,0.303030,0.253012,0.348837,0.150000,4.0,...,2.0,20.15,7.0,0.875000,0.983192,61.666667,61.333333,4.833333,5.500000,0.521008
2,273303,2629,2632,0.370430,0.375756,0.340690,0.278481,0.333333,0.225000,1.0,...,8.0,53.90,15.0,1.875000,1.861899,45.000000,49.000000,4.666667,6.666667,0.539683
3,273306,2630,2611,0.588560,0.202962,0.289960,0.280488,0.400000,0.166667,3.0,...,3.0,29.20,8.0,1.333333,1.366260,57.500000,55.166667,4.333333,7.833333,0.486667
4,273308,2614,2628,0.432232,0.305744,0.345338,0.219512,0.236842,0.204545,6.0,...,7.0,39.30,10.0,1.666667,1.673320,55.833333,50.500000,3.500000,4.333333,0.540230


In [169]:
df2 = pd.merge(df2, coming_games, on = ['match_id', 'home_id', 'away_id']).drop(['home_name', 'away_name', 'home_score', 'away_score', 'home_half_score',
       'away_half_score', 'league_id'],axis=1)

In [170]:
df2['h_mean_poss'] = df2['h_mean_poss']/100
df2['h_mean_poss2'] = df2['h_mean_poss2']/100
df2['a_mean_poss'] = df2['a_mean_poss']/100
df2['a_mean_poss2'] = df2['a_mean_poss2']/100

In [171]:
preds = best_model.predict_proba(df2.iloc[:,3:])

In [ ]:
preds # P(Home)    P(Tie)   P(Away)

results=str()
for i in range(len(preds)):
    results += f'{df2.match_id.values[i]},{preds[i][0]},{preds[i][2]},{preds[i][1]},'
    print(f'{df2.match_id.values[i]},{preds[i][0]},{preds[i][1]},{preds[i][2]}')

Results in proper format for Google Form:

In [ ]:
results